In [198]:
from data import NumericalField, CategoricalField, Iterator
from data import Dataset
from synthesizer import VGAN_generator, VGAN_discriminator
from synthesizer import V_Train
from utils import to_df,KL_Loss,mean_Loss
from random import choice

import pandas as pd
import numpy as np
import torch

import json

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import random
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import pandas as pd

In [326]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [199]:
VGAN_variable = {
	"batch_size":[64],
	"z_dim":[128],
	"gen_hidden_dim":[200],
	"gen_num_layers":[3],
	"dis_hidden_dim":[200],
	"dis_num_layers":[3],
	"dis_lstm_dim":[200],
	"lr":[0.0002],
	"noise":[0.1]
}

In [200]:
def parameter_search(Model):
	parameters = {}
	if Model == "VGAN":
		variable = VGAN_variable

	# 创建一个存 parameter的dictionary,随机选择一个parameter
	for param in variable.keys():
		parameters[param] = choice(variable[param])
	return parameters

In [201]:
config = { 
        "name": "adult_fd_test",
		"train": "adult_train_sample.csv",	
		"sample": "adult_train_sample.csv",
		"gmm_cols":[],
		"normalize_cols":[0,2,4,10,11,12],
		"one-hot_cols":[1,3,5,6,7,8,9,13,14],
		"ordinal_cols":[],
		"model": "VGAN",
		"n_epochs":20,
		"steps_per_epoch":10,
		"n_search": 5,
		"rand_search": "yes",
		"train_method":"VTrain"
	}


In [203]:
train = pd.read_csv(config["train"])
n_search = config["n_search"]
search = 1
ratio = 0.9
noise = 0.1
fields = []
col_type = []



for i, col in enumerate(list(train)):

    if i in config["normalize_cols"]:
        fields.append((col,NumericalField("normalize")))
        col_type.append("normalize")
    elif i in config["gmm_cols"]:
        fields.append((col, NumericalField("gmm", n=5)))
        col_type.append("gmm")
    elif i in config["one-hot_cols"]:
        fields.append((col, CategoricalField("one-hot", noise=noise)))
        col_type.append("one-hot")
    elif i in config["ordinal_cols"]:
        fields.append((col, CategoricalField("dict")))
        col_type.append("ordinal")
    else:
        fields.append((col, CategoricalField("binary",noise=noise)))
        col_type.append("binary")


# print("train row : {}".format(len(trn)))
# print("sample row: {}".format(len(samp)))




In [204]:
trn, samp = Dataset.split(
    fields = fields,
    path = ".",
    train = config["train"],
    validation = config["sample"],
    format = "csv",
    valid_ratio=ratio
)


In [205]:
fields

[('age', <data.field.NumericalField at 0x226c26d8688>),
 ('workclass', <data.field.CategoricalField at 0x226c26d8d88>),
 ('fnlwgt', <data.field.NumericalField at 0x226c26d8808>),
 ('education', <data.field.CategoricalField at 0x226c26d8a88>),
 ('education-num', <data.field.NumericalField at 0x226c26d8348>),
 ('marital-status', <data.field.CategoricalField at 0x226c260e2c8>),
 ('occupation', <data.field.CategoricalField at 0x226c260e408>),
 ('relationship', <data.field.CategoricalField at 0x226c260e3c8>),
 ('race', <data.field.CategoricalField at 0x226c260e0c8>),
 ('sex', <data.field.CategoricalField at 0x226c26d8a08>),
 ('capital-gain', <data.field.NumericalField at 0x226c260e348>),
 ('capital-loss', <data.field.NumericalField at 0x226c260e248>),
 ('hours-per-week', <data.field.NumericalField at 0x224f9b1cf88>),
 ('native-country', <data.field.CategoricalField at 0x224f9b1ce88>),
 ('income', <data.field.CategoricalField at 0x224f9b1cf48>)]

In [206]:
trn.learn_convert()
samp.learn_convert()

In [207]:
model = config["model"]
train_method = config["train_method"]


param = parameter_search(config["model"])
train_it, sample_it = Iterator.split(
    batch_size = param["batch_size"],
    train = trn,
    validation = samp,
    sort_key = None,
    shuffle = True,
    labels = None,
    square = False,
    pad = None
)
x_dim = train_it.data.shape[1]
col_dim = trn.col_dim
col_ind = trn.col_ind

In [208]:
c_dim = 0
condition = False
sample_times = 1
KL = True
GPU = torch.cuda.is_available()

In [209]:
GPU

True

In [324]:
def fd_calculated(df_fake,fd_model,y_fake):

    if fd_model == "simple":

        Input = df_fake["education-num"].astype('float32')
        Output = df_fake["capital-gain"]
        predicted = FD_simple.predict(Input).flatten()

        fd = sum(abs(predicted-Output))/y_fake.shape[0]

        return fd

    if fd_model == "strong":
        
        Input = pd.get_dummies(df_fake["relationship"])
        Input = np.array(pd.DataFrame(Input,columns=['Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried',
       'Wife']).fillna(0))
       
        Output = df_fake["marital-status"].astype('category').cat.codes

        predicted = np.argmax(FD_strong.predict(Input),axis = 1) 
        fd = np.sum(Output != predicted)

        return fd

    if fd_model == "complex":
        Input = df_fake[["education-num","age","hours-per-week"]].astype('int')
        Output = np.array(pd.get_dummies(df_fake["income"]))
        predicted = np.argmax(FD_complex.predict(Input),axis = 1)
        fd = np.sum(Output != predicted)

        return fd 



        



In [223]:
def V_Train(t, path, sampleloader, G, D, fd_model,
epochs, lr, dataloader, z_dim, dataset, 
col_type, sample_times, itertimes = 100, 
steps_per_epoch = None, GPU=False, KL=True, method = "ITS"):

    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    
    if GPU:
        G.cuda()
        D.cuda()
    G.GPU = True
       
    D_optim = optim.Adam(D.parameters(), lr=lr, weight_decay=0.00001)
    G_optim = optim.Adam(G.parameters(), lr=lr, weight_decay=0.00001)

    # the default # of steps is the # of batches.

    if steps_per_epoch is None:
        steps_per_epoch = len(dataloader)

    for epoch in range(epochs):
        it = 0
        print("-----------Epoch {}-----------".format(epoch))
        while it < steps_per_epoch:
            
            # batch 128, x_real.shape = [128,105]
            for x_real in dataloader:
                if GPU:
                    x_real = x_real.cuda()
                ''' train Discriminator '''
                z = torch.randn(x_real.shape[0], z_dim)
                
                if GPU:
                    z = z.cuda()   
                x_fake = G(z)
                
                y_real = D(x_real)
                y_fake = D(x_fake)

                if method == "ITS" or method == "fd":


                    df_fake = to_df(x_fake,dataset)
                    
                    D_fd = fd_calculated(df_fake,fd_model,y_fake)

                else:
                    D_fd = 0


                # 生成 0 和 1，避免  Discriminator 对 Generator 压制
                # D_Loss = -(torch.mean(y_real) - torch.mean(y_fake)) # wgan loss
                fake_label = torch.zeros(y_fake.shape[0], 1)
                real_label = np.ones([y_real.shape[0], 1])
                # Avoid the suppress of Discriminator over Generator
                real_label = real_label * 0.7 + np.random.uniform(0, 0.3, real_label.shape)
                real_label = torch.from_numpy(real_label).float()
                
                if GPU:

                    fake_label = fake_label.cuda()
                    real_label = real_label.cuda()

                    
                D_Loss1 = F.binary_cross_entropy(y_real, real_label)
                D_Loss2 = F.binary_cross_entropy(y_fake, fake_label)

                D_Loss = D_Loss1 + D_Loss2 + D_fd

                #print("D Loss: {} | fd: {} ".format(D_Loss, D_fd))
                
                G_optim.zero_grad()
                D_optim.zero_grad()
                D_Loss.backward()
                D_optim.step()

                ''' train Generator '''
                # 生成 一批noisy
                z = torch.randn(x_real.shape[0], z_dim)
                if GPU:
                    z = z.cuda()
                
                x_fake = G(z)
                y_fake = D(x_fake)

                if method == "ITS" or "fd":

                    df_fake = to_df(x_fake,dataset)
                    
                    G_fd = fd_calculated(df_fake,fd_model,y_fake)

                else:
                    G_fd = 0

                
                

                real_label = torch.ones(y_fake.shape[0], 1)
                if GPU:
                    real_label = real_label.cuda()

                G_Loss1 = F.binary_cross_entropy(y_fake, real_label)

                if method == "ITS" or method == "mean":
                    G_Loss2 = mean_Loss(x_fake, x_real, col_type, dataset.col_dim)
                else:
                    G_Loss2 = 0

                if KL:
                    KL_loss = KL_Loss(x_fake, x_real, col_type, dataset.col_dim)
                    G_Loss = G_Loss1 + KL_loss + G_fd + G_Loss2
                    #print("G Loss: {} | Origin: {} | KL: {} | fd: {} | mean: {} ".format(G_Loss, G_Loss1, KL_loss, G_fd, G_Loss2))
                    #print("G Loss: {} | fd: {} ".format(G_Loss,G_fd))
                else:
                    G_Loss = G_Loss1 + G_fd + G_Loss2
                

                G_optim.zero_grad()
                D_optim.zero_grad()
                G_Loss.backward()
                G_optim.step()

                it += 1

                if it%itertimes == 0:
                    print("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                if it >= steps_per_epoch:
                    G.eval()
                    for time in range(sample_times):
                        sample_data = None
                        for x_real in sampleloader:
                            z = torch.randn(x_real.shape[0], z_dim)
                            if GPU:
                                z = z.cuda()
                            x_fake = G(z)
                            samples = x_fake
                            samples = samples.reshape(samples.shape[0], -1)
                            samples = samples[:,:dataset.dim]
                            samples = samples.cpu()
                            sample_table = dataset.reverse(samples.detach().numpy())
                            df = pd.DataFrame(sample_table,columns=dataset.columns)
                            if sample_data is None:
                                sample_data = df
                            else:
                                sample_data = sample_data.append(df)
                        sample_data.to_csv(path+'sample_data_{}_{}_{}.csv'.format(t,epoch,time), index = None)
                   # if GPU:
                   #     G.cuda()
                    #    G.GPU = True
                    G.train()
                    break
    return G,D

In [ ]:
gen = VGAN_generator(param["z_dim"], param["gen_hidden_dim"], x_dim, 
        param["gen_num_layers"], col_type, col_ind, condition=condition,c_dim=c_dim)

dis = VGAN_discriminator(x_dim, param["dis_hidden_dim"], 
        param["dis_num_layers"],condition,c_dim)

V_Train(search, "generated/test/", sample_it, gen, dis, "strong",
        700, param["lr"], train_it, param["z_dim"], trn, col_type, 
        sample_times,itertimes = 100, steps_per_epoch = config["steps_per_epoch"], GPU=GPU, KL=KL)

In [272]:
def generation(fd, path,method):

    gen = VGAN_generator(param["z_dim"], param["gen_hidden_dim"], x_dim, 
        param["gen_num_layers"], col_type, col_ind, condition=condition,c_dim=c_dim)

    dis = VGAN_discriminator(x_dim, param["dis_hidden_dim"], 
        param["dis_num_layers"],condition,c_dim)

    V_Train(search, path, sample_it, gen, dis, fd,
        700, param["lr"], train_it, param["z_dim"], trn, col_type, 
        sample_times,itertimes = 100, steps_per_epoch = config["steps_per_epoch"], GPU=GPU, KL=KL, method = method)
    

In [273]:
generation("strong", "generated/strong/ITS/","ITS")

-----------Epoch 0-----------
-----------Epoch 1-----------
-----------Epoch 2-----------
-----------Epoch 3-----------
-----------Epoch 4-----------
-----------Epoch 5-----------
-----------Epoch 6-----------
-----------Epoch 7-----------
-----------Epoch 8-----------
-----------Epoch 9-----------
-----------Epoch 10-----------
-----------Epoch 11-----------
-----------Epoch 12-----------
-----------Epoch 13-----------
-----------Epoch 14-----------
-----------Epoch 15-----------
-----------Epoch 16-----------
-----------Epoch 17-----------
-----------Epoch 18-----------
-----------Epoch 19-----------
-----------Epoch 20-----------
-----------Epoch 21-----------
-----------Epoch 22-----------
-----------Epoch 23-----------
-----------Epoch 24-----------
-----------Epoch 25-----------
-----------Epoch 26-----------
-----------Epoch 27-----------
-----------Epoch 28-----------
-----------Epoch 29-----------
-----------Epoch 30-----------
-----------Epoch 31-----------
-----------Epoch 3

In [274]:
generation("strong", "generated/strong/fd/","fd")

-----------Epoch 0-----------
-----------Epoch 1-----------
-----------Epoch 2-----------
-----------Epoch 3-----------
-----------Epoch 4-----------
-----------Epoch 5-----------
-----------Epoch 6-----------
-----------Epoch 7-----------
-----------Epoch 8-----------
-----------Epoch 9-----------
-----------Epoch 10-----------
-----------Epoch 11-----------
-----------Epoch 12-----------
-----------Epoch 13-----------
-----------Epoch 14-----------
-----------Epoch 15-----------
-----------Epoch 16-----------
-----------Epoch 17-----------
-----------Epoch 18-----------
-----------Epoch 19-----------
-----------Epoch 20-----------
-----------Epoch 21-----------
-----------Epoch 22-----------
-----------Epoch 23-----------
-----------Epoch 24-----------
-----------Epoch 25-----------
-----------Epoch 26-----------
-----------Epoch 27-----------
-----------Epoch 28-----------
-----------Epoch 29-----------
-----------Epoch 30-----------
-----------Epoch 31-----------
-----------Epoch 3

In [275]:
generation("strong", "generated/strong/mean/","mean")

-----------Epoch 0-----------
-----------Epoch 1-----------
-----------Epoch 2-----------
-----------Epoch 3-----------
-----------Epoch 4-----------
-----------Epoch 5-----------
-----------Epoch 6-----------
-----------Epoch 7-----------
-----------Epoch 8-----------
-----------Epoch 9-----------
-----------Epoch 10-----------
-----------Epoch 11-----------
-----------Epoch 12-----------
-----------Epoch 13-----------
-----------Epoch 14-----------
-----------Epoch 15-----------
-----------Epoch 16-----------
-----------Epoch 17-----------
-----------Epoch 18-----------
-----------Epoch 19-----------
-----------Epoch 20-----------
-----------Epoch 21-----------
-----------Epoch 22-----------
-----------Epoch 23-----------
-----------Epoch 24-----------
-----------Epoch 25-----------
-----------Epoch 26-----------
-----------Epoch 27-----------
-----------Epoch 28-----------
-----------Epoch 29-----------
-----------Epoch 30-----------
-----------Epoch 31-----------
-----------Epoch 3

In [276]:
generation("strong", "generated/strong/origin/","origin")

-----------Epoch 0-----------
-----------Epoch 1-----------
-----------Epoch 2-----------
-----------Epoch 3-----------
-----------Epoch 4-----------
-----------Epoch 5-----------
-----------Epoch 6-----------
-----------Epoch 7-----------
-----------Epoch 8-----------
-----------Epoch 9-----------
-----------Epoch 10-----------
-----------Epoch 11-----------
-----------Epoch 12-----------
-----------Epoch 13-----------
-----------Epoch 14-----------
-----------Epoch 15-----------
-----------Epoch 16-----------
-----------Epoch 17-----------
-----------Epoch 18-----------
-----------Epoch 19-----------
-----------Epoch 20-----------
-----------Epoch 21-----------
-----------Epoch 22-----------
-----------Epoch 23-----------
-----------Epoch 24-----------
-----------Epoch 25-----------
-----------Epoch 26-----------
-----------Epoch 27-----------
-----------Epoch 28-----------
-----------Epoch 29-----------
-----------Epoch 30-----------
-----------Epoch 31-----------
-----------Epoch 3

In [327]:
generation("complex", "generated/complex/ITS/","ITS")

-----------Epoch 0-----------
-----------Epoch 1-----------
-----------Epoch 2-----------
-----------Epoch 3-----------
-----------Epoch 4-----------
-----------Epoch 5-----------
-----------Epoch 6-----------
-----------Epoch 7-----------
-----------Epoch 8-----------
-----------Epoch 9-----------
-----------Epoch 10-----------
-----------Epoch 11-----------
-----------Epoch 12-----------
-----------Epoch 13-----------
-----------Epoch 14-----------
-----------Epoch 15-----------
-----------Epoch 16-----------
-----------Epoch 17-----------
-----------Epoch 18-----------
-----------Epoch 19-----------
-----------Epoch 20-----------
-----------Epoch 21-----------
-----------Epoch 22-----------
-----------Epoch 23-----------
-----------Epoch 24-----------
-----------Epoch 25-----------
-----------Epoch 26-----------
-----------Epoch 27-----------
-----------Epoch 28-----------
-----------Epoch 29-----------
-----------Epoch 30-----------
-----------Epoch 31-----------
-----------Epoch 3

In [328]:
generation("complex", "generated/complex/mean/","mean")

-----------Epoch 0-----------
-----------Epoch 1-----------
-----------Epoch 2-----------
-----------Epoch 3-----------
-----------Epoch 4-----------
-----------Epoch 5-----------
-----------Epoch 6-----------
-----------Epoch 7-----------
-----------Epoch 8-----------
-----------Epoch 9-----------
-----------Epoch 10-----------
-----------Epoch 11-----------
-----------Epoch 12-----------
-----------Epoch 13-----------
-----------Epoch 14-----------
-----------Epoch 15-----------
-----------Epoch 16-----------
-----------Epoch 17-----------
-----------Epoch 18-----------
-----------Epoch 19-----------
-----------Epoch 20-----------
-----------Epoch 21-----------
-----------Epoch 22-----------
-----------Epoch 23-----------
-----------Epoch 24-----------
-----------Epoch 25-----------
-----------Epoch 26-----------
-----------Epoch 27-----------
-----------Epoch 28-----------
-----------Epoch 29-----------
-----------Epoch 30-----------
-----------Epoch 31-----------
-----------Epoch 3

In [16]:
fd_data = pd.read_csv("modified_adult_train.csv")
A = fd_data["education-num"]
B = fd_data["capital-gain"]
def build_model_simple():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[1]),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mae',
                optimizer=optimizer)
  return model


FD_simple = build_model_simple()

FD_simple.fit(
  A, B,epochs=100,batch_size=256)

Epoch 1/100
118/118 [==============================] - 1s 4ms/step - loss: 5041.4150
Epoch 2/100
118/118 [==============================] - 0s 4ms/step - loss: 4996.3340
Epoch 3/100
118/118 [==============================] - 0s 4ms/step - loss: 4906.3560
Epoch 4/100
118/118 [==============================] - 0s 3ms/step - loss: 4756.9175
Epoch 5/100
118/118 [==============================] - 0s 3ms/step - loss: 4533.4131
Epoch 6/100
118/118 [==============================] - 0s 3ms/step - loss: 4221.1689
Epoch 7/100
118/118 [==============================] - 0s 3ms/step - loss: 3806.0266
Epoch 8/100
118/118 [==============================] - 0s 3ms/step - loss: 3274.1841
Epoch 9/100
118/118 [==============================] - 0s 4ms/step - loss: 2610.9822
Epoch 10/100
118/118 [==============================] - 0s 3ms/step - loss: 1801.2662
Epoch 11/100
118/118 [==============================] - 0s 3ms/step - loss: 836.0342
Epoch 12/100
118/118 [==============================] - 0s 3ms/s

In [249]:
fd_data = pd.read_csv("adult_train.csv")
A = fd_data["relationship"].astype('category').cat.codes
B = fd_data["marital-status"]


A_one = np.array(pd.get_dummies(fd_data["relationship"]))
B_one = np.array(pd.get_dummies(fd_data["marital-status"]))
 

def build_model_strong():
  model = keras.Sequential([
    layers.Dense(8, input_dim=6, activation='relu'),
	  layers.Dense(7, activation='softmax'),
  ])

  optimizer = keras.optimizers.Adam(learning_rate=0.001)

  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
               optimizer=optimizer, metrics=['accuracy'])
  return model


FD_strong = build_model_strong()

FD_strong.fit(
  A_one, B_one,epochs=30,batch_size=256)


np.argmax(FD_strong.predict(A_one),axis = 1) 

Epoch 1/30
118/118 [==============================] - 0s 2ms/step - loss: 1.8298 - accuracy: 0.2549
Epoch 2/30
118/118 [==============================] - 0s 2ms/step - loss: 1.4566 - accuracy: 0.5747
Epoch 3/30
118/118 [==============================] - 0s 2ms/step - loss: 1.0935 - accuracy: 0.7678
Epoch 4/30
118/118 [==============================] - 0s 2ms/step - loss: 0.8349 - accuracy: 0.7678
Epoch 5/30
118/118 [==============================] - 0s 2ms/step - loss: 0.7018 - accuracy: 0.7678
Epoch 6/30
118/118 [==============================] - 0s 2ms/step - loss: 0.6372 - accuracy: 0.8020
Epoch 7/30
118/118 [==============================] - 0s 2ms/step - loss: 0.6021 - accuracy: 0.8053
Epoch 8/30
118/118 [==============================] - 0s 2ms/step - loss: 0.5819 - accuracy: 0.8053
Epoch 9/30
118/118 [==============================] - 0s 2ms/step - loss: 0.5684 - accuracy: 0.8053
Epoch 10/30
118/118 [==============================] - 0s 2ms/step - loss: 0.5578 - accuracy: 0.8053

array([4, 2, 2, ..., 2, 4, 2], dtype=int64)

In [303]:
Input = fd_data[["education-num","age","hours-per-week"]]
Output = fd_data["income"].astype('category').cat.codes
Output_dummy = np.array(pd.get_dummies(fd_data["income"]))

In [304]:
def build_model_complex():
  model = keras.Sequential([
    layers.Dense(64, input_dim=3, activation='relu'),
    layers.Dense(8, activation='relu'),
	  layers.Dense(2, activation='softmax'),
  ])

  optimizer = keras.optimizers.Adam(learning_rate=0.001)

  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
               optimizer=optimizer, metrics=['accuracy'])
  return model


FD_complex = build_model_complex()

FD_complex.fit(
  Input, Output_dummy,epochs=30,batch_size=256)


Epoch 1/30
118/118 [==============================] - 1s 3ms/step - loss: 0.6408 - accuracy: 0.7258
Epoch 2/30
118/118 [==============================] - 0s 2ms/step - loss: 0.5623 - accuracy: 0.7509
Epoch 3/30
118/118 [==============================] - 0s 3ms/step - loss: 0.5436 - accuracy: 0.7516
Epoch 4/30
118/118 [==============================] - 0s 3ms/step - loss: 0.5265 - accuracy: 0.7559
Epoch 5/30
118/118 [==============================] - 0s 2ms/step - loss: 0.5147 - accuracy: 0.7587
Epoch 6/30
118/118 [==============================] - 0s 2ms/step - loss: 0.5064 - accuracy: 0.7638
Epoch 7/30
118/118 [==============================] - 0s 3ms/step - loss: 0.4975 - accuracy: 0.7655
Epoch 8/30
118/118 [==============================] - 0s 3ms/step - loss: 0.4925 - accuracy: 0.7690
Epoch 9/30
118/118 [==============================] - 0s 3ms/step - loss: 0.4860 - accuracy: 0.7679
Epoch 10/30
118/118 [==============================] - 0s 2ms/step - loss: 0.4786 - accuracy: 0.7722

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [308]:
np.argmax(FD_complex.predict(Input),axis = 1) 

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [321]:
Input = np.array(fd_data[["education-num","age","hours-per-week"]])
Output = np.array(pd.get_dummies(fd_data["income"]))
print(FD_complex.predict(Input))

[[0.8081473  0.19185272]
 [0.81772405 0.18227598]
 [0.7122966  0.2877034 ]
 ...
 [0.36441836 0.6355817 ]
 [0.45115218 0.54884785]
 [0.34909508 0.6509049 ]]
